<a href="https://colab.research.google.com/github/lab30041954/ML_IESE_Course/blob/main/%5BML-03%5D%20Decision%20trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [ML-03] Decision trees

## What is a decision tree?

A **decision tree** is a collection of **decision nodes**, connected by branches, extending downwards from the **root node**, until terminating in the **leaf nodes**. The typical visualization of a decision tree puts the root on top and the leaves at the bottom, as in Figures 1 and 2 below.

Decision trees can be used for both regression and classification purposes. A decision tree creates a partition of the data set into a collection of subsets, one for each leaf. In a predictive model based on a decision tree, the prediction is the same for all the data units from the same leaf. More specifically:


* In a **decision tree regressor**, the predicted target value is the average target value in that leaf. 

* In a **decision tree classifier**, a predicted probability class is the proportion of units from the class in that leaf. Under the **default prediction rule**, the predicted class is the one that occurs more frequently.

## Decision trees in scikit-learn

There are various ways to train a decision tree model. The top popular one is the **CART** (Classification And Regression Trees) algorithm. In scikit-learn, the subpackage `tree` provides the estimator classes `DecisionTreeRegressor()` and `DecisionTreeClassifier()`, both based on CART.

At every decision node, there is a **split**, based on one of the features and a cutoff value. CART chooses at every node the **optimal split**, that minimizes the **loss**. In decision tree regressors, as in linear regression, the default loss is the **mean square error** (MSE). Other choices are available through the parameter `criterion`. 

![](https://raw.githubusercontent.com/lab30041954/Figures/main/tree1.png)

Figure 1 shows a decision tree regressor, developed to predict the price of a house (see lecture ML-05). At every node, we find the number of data units (`samples`), the MSE (`squared_error`) and the predicted price (`value`), which is the average price in that leaf. The tree is optimal (meaning that it provides the minimum MSE) among those satisfying the conditions set by the arguments of `DecisionTreeRegressor()`. For every possible split, CART calculates the loss as the weighted average of the losses at the two branches, choosing the split that leads to the minimum loss. 

![](https://raw.githubusercontent.com/lab30041954/Figures/main/tree2.png)

In scikit-learn decision tree classifiers, the default loss function is **Gini impurity measure**. Nevertheless, to present a more consistent approach to classification models along this course, we use everywhere the **average cross-entropy** (see lecture ML-01), which, in the class `DecisionTreeClassifier()`, is specified with the argument `criterion='entropy'`. Figure 2 shows a decision tree intended to be used in a spam filter (see the example of this lecture). At every leaf, you find the number of units (`samples`), the average cross-entropy (`entropy`) and the number of negative and positive units in that leaf (`value`), in alphabetical order, so negative comes first. As the predicted class probabilities in a leaf, the model takes the class proportions in that leaf. In a binary setting, we can say that the predicted score for a data unit is the proportion of positive units in the leaf where that unit falls. The tree is optimal in the sense that the overall average cross-entropy (the weighted average of the cross-entropies at the leaf nodes) is minimum.

The following exercises can help you to understand how this works:

1. In Figure 1, the initial MSE is equal to 134,776.142. Calculate the final MSE as the weighted average of the MSE's in the four leaf nodes. Translate the MSE reduction into a R-squared value (what the method `.score()` would return for this model).

2. Check that the initial cross-entropy in Figure 2 is indeed 0.967. What is the final entropy, after the three splits? Remember that scikit-learn uses binary logs to calculate the cross-entropy. In Python, you get the binary log as `math.log(x, 2)`.

3. In Figure 2, suppose that the split in the root node has already been performed. The second split will be the one that brings a greater reduction of the cross-entropy. Is this the leftt one (`x[15] <= 0.135`) or the right one (`x[26] <= 0.08`)?  

## Controlling the growth of the tree

The predictive models based on decision trees are prone to **overfitting**. Even with a moderate number of features, a tree model whose growth is not stopped can lead to a complex model with overfitting problems. In scikit-learn, the classes `DecisionTreeRegressor()` and `DecisionTreeClassifier()` have several parameters for controlling the growth of the tree: `max_depth`, `max_leaf_nodes`, `min_samples_split`, `min_samples_leaf`, `min_impurity_decrease`, etc. Only the first two will appear in this course.

To obtain the tree of Figure 1, we would use:

```
from sklearn.tree import DecisionTreeRegressor
reg = DecisionTreeRegressor(max_depth=2)
reg.fit(X, y)
```

Then, we would plot the tree with:

```
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
plt.figure(figsize=(13,7))
plot_tree(reg, fontsize=16);
```

To obtain the tree of Figure 2, we would use:

```
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(criterion='entropy', max_depth=2)
```


## Feature importance

One of the advantages of decision tree models is that it is very easy to get a report on the **feature importance**. The importance of a feature is computed as the contribution of that feature to the total loss reduction achieved by the model. This will be shown in the example of this lecture.

## Example - The spam filter

### Introduction

A **spam filter** is an AI agent which classifies e-mail messages as either spam or non-spam, based on a collection of **numeric features** such as the frequency of certain words or characters. In a spam filter, the **false positive rate**, that is, the proportion of non-spam messages wrongly classified as spam, must be very low.

The data of this example were gathered at Hewlett-Packard by merging: (a) a collection of spam e-mail from the company postmaster and the individuals who had filed spam, and (b) a collection of non-spam e-mail, extracted from filed work and personal e-mail. The proportions of spam and legal e-mail in this data set were arbitrary, not based in the proportions found in the real world.

### The data set

The file `spam.csv` contains data on 4,601 e-mail messages. Among these messages, 1,813 have been classified as spam. Every row in the data set corresponds to an e-mail message. The columns are:

* 48 numeric features whose names start with `word_`, followed by a word. They indicate the frequency, in percentage scale, with which that word appears in the message. Example: for a particular message, a value 0.21 for `word_make` means that 0.21% of the words in the message match the word 'make'.

* 3 numeric features indicating, respectively, the average length of uninterrupted sequences of capital letters (`cap_ave`), the length of the longest uninterrupted sequence of capital letters (`cap_long`) and the total number of capital letters in the message (`cap_total`).

* A dummy indicating whether that e-mail message is spam (`spam`).

Source: T Hastie, R Tibshirani & JH Friedman (2001), *The Elements of Statistical Learning*, Springer.


### Questions

Q1. Develop a spam filter based on a **decision tree** classifier with **maximum depth** 2 and evaluate its performance. 

Q2. Repeat the exercise allowing a maximum depth of 3.

Q3. Repeat the exercise allowing a maximum depth of 4.

Q4. Repeat the exercise allowing a maximum depth of 5.

Q5. Among the features available in this data set, which ones are more relevant for filtering spam?

### Importing the data

As in the preceding lectures, we use the Pandas funcion `read_csv()` to import the data from our GitHub repository. Since the data set does not include an identifier for the email messages, we allow Pandas to create a `RangeIndex`.

In [1]:
import pandas as pd
path = 'https://raw.githubusercontent.com/lab30041954/Data/main/'
df = pd.read_csv(path + 'spam.csv')

### Exploring the data

The dimensions of the data set are those expected from the description presented above.

In [2]:
df.shape

(4601, 52)

We also take a look at the first rows with the method `.head()`.

In [3]:
df.head()

,word_make,word_address,word_all,word_3d,word_our,word_over,word_remove,word_internet,word_order,word_mail,...,word_original,word_project,word_re,word_edu,word_table,word_conference,cap_ave,cap_long,cap_total,spam
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.0,0.00,0.00,0.0,0.0,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.0,0.00,0.00,0.0,0.0,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.12,0.0,0.06,0.06,0.0,0.0,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.0,0.00,0.00,0.0,0.0,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.0,0.00,0.00,0.0,0.0,3.537,40,191,1


Everything looks right. We also check the **spam rate** in this data set, which agrees with the description (1,813/4,601). Note that, as explained of the introduction above, this spam rate is not representative of the actual spam activity at the company.

In [4]:
round(df['spam'].mean(), 3)

np.float64(0.394)

### Target vector and features matrix

To use scikit-learn to obtain our decision tree classifiers, we create a **target vector** and a **features matrix**. The target vector is the last column (`spam`) and the features matrix integrates the other columns.

In [5]:
y = df['spam']
X = df.drop(columns='spam')

### Q1. Decision tree classifier (max depth = 2)

To develop our decision tree classifiers, we use the **estimator class** `DecisionTreeClassifier()` from the scikit-learn subpackage `tree`. We import this class as in the preceding lectures.

In [6]:
from sklearn.tree import DecisionTreeClassifier

We instantiate a first estimator from this class, setting `max_depth=2`, which limits the **depth**, that is, the length of the longest branch of the tree. As explained above, we use the **cross-entropy** loss function. Since this is not the default in `DecisionTreeClassifier()`, we include the argument `criterion='entropy'`.

In [7]:
clf1 = DecisionTreeClassifier(criterion='entropy', max_depth=2)

The method `.fit()` finds the optimal decision tree under this specification. Figure 2 is a visualization of this tree. It has four leaves and uses only three of the 51 features available.

In [8]:
clf1.fit(X, y)

,criterion,'entropy'
,splitter,'best'
,max_depth,2
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


The method `.score()` gives us the **accuracy** of this model.

In [9]:
round(clf1.score(X, y), 3)

0.834

An 83.4% accuracy looks promising for a spam filter, but we have been warned about the false positive rate. Also, we know that the training data are a mix of spam and legal mail in arbitrary proportions. Therefore, this 83.4% does not apply to the real world, it is just a weighted average of the accuracy of the model with spam mail and the accuracy with legal mail, but with arbitrary weights. 

So, we take a closer look at the predictions of this model, by means of the **confusion matrix**. The code is the same as in the preceding lectures.

In [10]:
y_pred1 = clf1.predict(X)
from sklearn.metrics import confusion_matrix
conf1 = confusion_matrix(y, y_pred1)
conf1

array([[2575,  213],
       [ 549, 1264]])

The counts in the first row of this matrix correspond to the negative data units (the legal mail), while those in the second row correspond to the positive units (the spam mail). The model performs better with the legal mail than with the spam mail. As in lecture ML-01, we use the **true positive rate** and the **false positive rate**, to evaluate this model.

In [11]:
import numpy as np
tp1 = conf1[1, 1]/sum(conf1[1, :])
fp1 = conf1[0, 1]/np.sum(conf1[0, :])
round(tp1, 3), round(fp1, 3)

(np.float64(0.697), np.float64(0.076))

We would like to improve these statistics. Since the model that we are using is supersimple, an obvious approach is to increase the parameter `max_depth`. Note that a decision tree with depth 2 uses at most three features.

### Q2. Decision tree classifier (max depth = 3)

We instantiate a second decision tree classifier, now with `max_depth=3`.

In [12]:
clf2 = DecisionTreeClassifier(criterion='entropy', max_depth=3)
clf2.fit(X, y)

,criterion,'entropy'
,splitter,'best'
,max_depth,3
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


Right now, we have two estimators from the class `DecisionTreeClassifier()`, namely `clf1` and `clf2`, both trained on our data. The confusion matrix gets better, specially the false negatives:

In [13]:
y_pred2 = clf2.predict(X)
conf2 = confusion_matrix(y, y_pred2)
conf2

array([[2637,  151],
       [ 560, 1253]])

The true positive rate and the false positive rate are, now:

In [14]:
tp2 = conf2[1, 1]/sum(conf2[1, :])
fp2 = conf2[0, 1]/sum(conf2[0, :])
round(tp2, 3), round(fp2, 3)

(np.float64(0.691), np.float64(0.054))

The additional branching has, at most, added four decision nodes, so this decision tree may be using, at most, seven different features.

### Q3. Decision tree classifier (max depth = 4)

To address question Q3, we allow the tree some extra growth, setting `max_depth=4`.

In [15]:
clf3 = DecisionTreeClassifier(criterion='entropy', max_depth=4)
clf3.fit(X, y)

,criterion,'entropy'
,splitter,'best'
,max_depth,4
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


Again, we examine the confusion matrix.

In [16]:
y_pred3 = clf3.predict(X)
conf3 = confusion_matrix(y, y_pred3)
conf3

array([[2483,  305],
       [ 213, 1600]])

Though the true positive rate is getting attractive, the false positive rate is still too high. This tree may be using 15 features.

In [17]:
tp3 = conf3[1, 1]/sum(conf3[1, :])
fp3 = conf3[0, 1]/sum(conf3[0, :])
round(tp3, 3), round(fp3, 3)

(np.float64(0.883), np.float64(0.109))

### Q4. Decision tree classifier (max depth = 5)

As the final assault, we train a new decision tree classifier, with `max_depth=5`.

In [18]:
clf4 = DecisionTreeClassifier(criterion='entropy', max_depth=5)
clf4.fit(X, y)

,criterion,'entropy'
,splitter,'best'
,max_depth,5
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [19]:
y_pred4 = clf4.predict(X)
conf4 = confusion_matrix(y, y_pred4)
conf4

array([[2630,  158],
       [ 289, 1524]])

Now, the false positive rate gets better, while the true positive rate remains acceptable. We stop here, leaving an alternative approach for the homework section.

In [20]:
tp4 = conf4[1, 1]/sum(conf4[1, :])
fp4 = conf4[0, 1]/sum(conf4[0, :])
round(tp4, 3), round(fp4, 3)

(np.float64(0.841), np.float64(0.057))

### Q5. Feature relevance

In scikit-learn, we get, as a by-product, a ranking of the features by their importance. This is the attribute `.feature_importances_`. It is extracted as a 1D array, in which each term is the **importance** of one of the features. The importance is measured as the **percentage of loss reduction** due to the splits in which the feature is involved. A zero importance value means that the corresponding feature is not involved in any split, so it is not used by the decision tree.

We take a look at feature importance in the biggest of our trees. In spite of the allowance for depth 5, only 15 features are involved in the splits.

In [21]:
imp = clf4.feature_importances_
imp

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.35897513, 0.        , 0.        , 0.00302331,
       0.0133721 , 0.        , 0.        , 0.        , 0.        ,
       0.2042085 , 0.        , 0.        , 0.        , 0.        ,
       0.00739229, 0.        , 0.        , 0.0449096 , 0.12960026,
       0.0021334 , 0.05894863, 0.00362196, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.00406665, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.03431209, 0.        , 0.        , 0.11320898, 0.02222709,
       0.        ])

To prepare a better report, we transform this array into a Pandas series, with the feature names as the index.

In [22]:
feat_list = pd.Series(imp, index=df.columns[:51])

Finally, we filter out the non-relevant features and sort them by relevance.

In [23]:
feat_list[imp > 0].sort_values(ascending=False).round(3)

word_remove     0.359
word_free       0.204
word_hp         0.130
cap_ave         0.113
word_george     0.059
word_money      0.045
word_edu        0.034
cap_long        0.022
word_receive    0.013
word_your       0.007
word_1999       0.004
word_650        0.004
word_mail       0.003
word_hpl        0.002
dtype: float64

### Homework

1. Train a **logistic regression** classifier on the data of this example and compare its performance to that of the classifiers presented in this lecture.

2. Change the features matrix by: (a) dropping the three `cap_` features and (b) **binarizing** all the `word_` features, transforming every column into a dummy for the occurrence of the corresponding word, taking value 1 if the word occurs in the message and 0 otherwise. Based on this new features matrix, train two new spam filters, one based on a logistic regression model and the other one based on a decision tree model, using the binarized data set. Evaluate these new filters and compare them to those obtained before.